In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("heart_rates(id).csv")
del df['id']

In [ ]:
df_enddate = pd.read_csv("patient_end_date.csv")

In [ ]:
import datetime

def fill_enddate(value):
    if pd.isna(value):
        return datetime.datetime(9999,12,31)
    else:
        return value

df = pd.merge(df, df_enddate, left_on='fitbit_id', right_on='patient_id', how='left')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['end_date'] = pd.to_datetime(df['end_date'])

df['end_date'] = df['end_date'].apply(fill_enddate)
del df['patient_id']
df.rename(columns={'fitbit_id':'patient_id'}, inplace=True)
df = df[df['timestamp'] <= df['end_date']]

In [ ]:
df_patient = pd.read_csv("all_patient.csv")
df = df[df['patient_id'].isin(df_patient['patient_id'])]

In [ ]:
df.sort_values(['patient_id', 'timestamp'], inplace=True)

In [ ]:
my_dict = {}
arr = df.values

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(arr)), miniters=1, mininterval=1):
    p = arr[i][0]
    if p not in my_dict:
        my_dict[p] = {}
    current_date = datetime.datetime(arr[i][1].year,arr[i][1].month,arr[i][1].day)
    hour = arr[i][1].hour
    if hour <= 3:
        current_date = current_date - datetime.timedelta(days=1)
    if current_date not in my_dict[p] :
        my_dict[p][current_date] = {}
        my_dict[p][current_date]['day'] = []
        my_dict[p][current_date]['bedtime'] = []
        my_dict[p][current_date]['morning'] = []
        my_dict[p][current_date]['afternoon'] = []
        my_dict[p][current_date]['evening'] = []
    my_dict[p][current_date]['day'].append(arr[i][2])
    if hour >= 22 or hour < 4:
        my_dict[p][current_date]['bedtime'].append(arr[i][2])
    elif 4 <= hour < 10:
        my_dict[p][current_date]['morning'].append(arr[i][2])
    elif 10 <= hour < 16:
        my_dict[p][current_date]['afternoon'].append(arr[i][2])
    elif 16 < hour < 22:
        my_dict[p][current_date]['evening'].append(arr[i][2])

In [ ]:
my_dict_all_day = {}

for p in my_dict:
    keys = sorted(my_dict[p])
    start_date = keys[0]
    end_date = keys[-1]

    now = start_date
    my_dict_all_day[p] = {}
    while now <= end_date:
        my_dict_all_day[p][now] = {}
        my_dict_all_day[p][now]['day'] = {}
        my_dict_all_day[p][now]['day']['mean'] = np.nan
        my_dict_all_day[p][now]['day']['median'] = np.nan
        my_dict_all_day[p][now]['day']['std'] = np.nan
        my_dict_all_day[p][now]['day']['cos_of_mean'] = np.nan

        my_dict_all_day[p][now]['bedtime'] = {}
        my_dict_all_day[p][now]['bedtime']['mean'] = np.nan
        my_dict_all_day[p][now]['bedtime']['std'] = np.nan

        my_dict_all_day[p][now]['morning'] = {}
        my_dict_all_day[p][now]['morning']['mean'] = np.nan
        my_dict_all_day[p][now]['morning']['std'] = np.nan

        my_dict_all_day[p][now]['afternoon'] = {}
        my_dict_all_day[p][now]['afternoon']['mean'] = np.nan
        my_dict_all_day[p][now]['afternoon']['std'] = np.nan

        my_dict_all_day[p][now]['evening'] = {}
        my_dict_all_day[p][now]['evening']['mean'] = np.nan
        my_dict_all_day[p][now]['evening']['std'] = np.nan
        now = now + datetime.timedelta(days=1)

In [ ]:
for p in my_dict:
    for day in my_dict[p]:
        for time in my_dict[p][day] : 
            my_dict_all_day[p][day][time]['mean'] = np.mean(my_dict[p][day][time])
            my_dict_all_day[p][day][time]['std'] = np.std(my_dict[p][day][time])
            if time == 'day':
                my_dict_all_day[p][day][time]['median'] = np.quantile(my_dict[p][day][time], 0.5)
                my_dict_all_day[p][day][time]['cos_of_mean'] = np.cos(my_dict_all_day[p][day][time]['mean'])

In [ ]:
my_list = []
for p in my_dict_all_day:
    for day in sorted(my_dict_all_day[p]):
        row = [p,day.date()]
        for time in ['day', 'morning', 'afternoon', 'evening', 'bedtime']:
            if time == 'day':
                row.append(my_dict_all_day[p][day][time]['mean'])
                row.append(my_dict_all_day[p][day][time]['median'])
                row.append(my_dict_all_day[p][day][time]['std'])
                row.append(my_dict_all_day[p][day][time]['cos_of_mean'])
            else:
                row.append(my_dict_all_day[p][day][time]['mean'])
                row.append(my_dict_all_day[p][day][time]['std'])
        my_list.append(row)

df = pd.DataFrame(my_list, columns=["patient_id","timestamp","day_hr_mean","day_hr_median","day_hr_std","day_hr_cos_of_mean", \
                                    "morning_hr_mean","morning_hr_std", \
                                    "afternoon_hr_mean", "afternoon_hr_std", \
                                    "evening_hr_mean","evening_hr_std", \
                                    "bedtime_hr_mean","bedtime_hr_std"])
df                  

In [ ]:
df.to_csv("feature_heart_rate.csv", index=False, encoding="utf-8-sig")